This notebook does the following:

Recommend tags based on Tag Application history, for user 43715

1) # Tag Applications
    * user-tag-#apps
    * order by #apps
    * weighted by total num apps


In [7]:
import pandas as pd
import numpy as np

In [4]:
# Connect to the database
import pymysql.cursors
import json

# Connect to the database
connection = pymysql.connect(host="127.0.0.1",
                             user="web",
                             password="atth1132",
                             db="ML3_mirror")

try:
    with connection.cursor() as cursor:
        # This query select rows from log_trailer_actions where the action is either TrailerModalLaunched or Recommendations Refreshed
        # These two actions indicate that a recommendation list is generated. 
        
        sql = """
        SELECT userId, movieId, tag, affect, tstamp
        FROM tag_events
        WHERE userId = 313322
        """ 
        
        cursor.execute(sql)
        result1 = cursor.fetchall()
            
finally: 
    connection.close()


In [5]:
user_tag_app = pd.DataFrame(list(result1), columns=['userId','movieId','tag','affect','tstamp'])

In [6]:
# Count the number of times user applied a tag on a certain movie. 
# If we take the sum, that will be the # times user applied this tag

user_tag_app2 = user_tag_app.groupby(['tag','movieId']).count().reset_index()
# user_tag_app2.drop(columns=['tstamp'],inplace=True)
user_tag_app2.rename(columns={'userId':"user_num_app"},inplace=True)
user_tag_app2.head()

,tag,movieId,user_num_app,affect,tstamp
0,Hayao Miyazaki,3000,1,1,1
1,Hayao Miyazaki,8253,1,1,1
2,dystopia,3000,1,1,1
3,nature,3000,1,1,1
4,nature vs. civilization,3000,1,1,1


In [11]:
# This block counts how many times a tag has been applied; 
# Used for weighting user tags

import pymysql.cursors
import json

# Connect to the database
connection = pymysql.connect(host="127.0.0.1",
                             user="web",
                             password="atth1132",
                             db="ML3_mirror")
try:
    with connection.cursor() as cursor:
            sql = "SELECT * FROM tag_movie"
            cursor.execute(sql)
            result = cursor.fetchall()
            
finally: 
    connection.close()


In [12]:
movie_tag_app = pd.DataFrame(list(result), columns=['movieId','tag','numApps','numPositive','numNeutral','numNegative','numDownvotes','score'])
movie_tag_app.head()

,movieId,tag,numApps,numPositive,numNeutral,numNegative,numDownvotes,score
0,1,2009 reissue in Stereoscopic 3-D,1,1,0,0,2,-1.0
1,1,3D,3,0,3,0,3,0.0
2,1,55 movies every kid should see--Entertainment ...,1,0,1,0,2,-1.0
3,1,action,1,0,1,0,1,0.0
4,1,action figure,1,0,1,0,2,-1.0


### Sum the scores of movie-tag which a user has applied to a movie. 
#### This is an naive approach that incorporates tag_quality into user_num_app


In [13]:
user_movie_tag_score = pd.merge(user_tag_app2, movie_tag_app, on=['movieId','tag'], how='left')


NameError: name 'user_tag_app2' is not defined

In [11]:
# Sum the scores of tags which the user has applied.
user_movie_tag_score_agg = user_movie_tag_score.groupby(["tag"]).sum().reset_index().sort_values(by="score",ascending=False)

# Choose 10 as score threshold
# Exclude tags with a score < 10
user_movie_tag_score_agg = user_movie_tag_score_agg.loc[user_movie_tag_score_agg["score"]>=10]

# Note that this is the tag score agg only for one user
user_movie_tag_score_agg

,tag,movieId,user_num_app,affect,tstamp,numApps,numPositive,numNeutral,numNegative,numDownvotes,score
0,Hayao Miyazaki,11253,2,2,2,98,79,19,0,10,88.0
1,dystopia,3000,1,1,1,26,19,7,0,5,21.0
2,nature,3000,1,1,1,21,14,7,0,1,20.0


#### Filter out tags that are not in the candidate set

In [5]:
# This block counts how many times a tag has been applied; 
# Used for weighting user tags

import pymysql.cursors
import json

# Connect to the database
connection = pymysql.connect(host="127.0.0.1",
                             user="web",
                             password="atth1132",
                             db="ML3_dev")
try:
    with connection.cursor() as cursor:
            sql = "SELECT tag, tagPrefCandidates FROM tag_data"
            cursor.execute(sql)
            result = cursor.fetchall()
            
finally: 
    connection.close()



In [10]:
tag_data = pd.DataFrame(list(result), columns=['tag','tagPrefCandidates'])
tag_candidates = tag_data[tag_data['tagPrefCandidates']==True]['tag'].tolist()

In [12]:
# Rank tags by user_num_app, then by scores
user_movie_tag_score_agg[user_movie_tag_score_agg['tag'].isin(tag_candidates)].sort_values(by=["user_num_app","score"],ascending=False).head(10)

,tag,movieId,user_num_app,affect,tstamp,numApps,numPositive,numNeutral,numNegative,numDownvotes,score
105,dystopia,109599,3,3,3,353.0,304.0,39.0,10.0,19.0,334.0
214,violence,61713,3,3,3,218.0,84.0,90.0,44.0,10.0,208.0
163,pixar,337419,3,3,3,124.0,75.0,44.0,5.0,3.0,121.0
215,violent,237654,3,3,3,91.0,36.0,23.0,32.0,6.0,85.0
71,cars,325142,3,3,3,30.0,16.0,14.0,0.0,7.0,23.0
79,classic,853,2,2,2,476.0,76.0,397.0,3.0,12.0,464.0
95,dark comedy,3154,2,2,2,382.0,333.0,41.0,8.0,25.0,357.0
84,computers,83034,2,2,2,173.0,151.0,22.0,0.0,8.0,165.0
208,time travel,13485,2,2,2,151.0,104.0,46.0,1.0,6.0,145.0
124,hackers,123208,2,2,2,86.0,71.0,11.0,4.0,5.0,81.0
